# **IMPORT THƯ VIỆN**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# **DỮ LIỆU**

In [ ]:
constant = pd.read_csv("statistic_constant.csv", index_col="N")
#data = pd.read_csv("test_P.csv", index_col="Ngày")
data = pd.read_csv("testX_R.csv")
data

FileNotFoundError: [Errno 2] No such file or directory: 'statistic_constant.csv'

# **XỬ LÝ KỊCH BẢN MẤT KIỂM SOÁT**

In [ ]:
# Xử lý các kịch bản mất kiểm soát
def xu_ly_kich_ban_mat_kiem_soat(diem_gia_tri, UCL, LCL, mean, sigma, diem_mat_kiem_soat):
  # Kịch bản 1: 1 điểm vượt ngoài UCL hoặc LCL
  for idx, val in enumerate(diem_gia_tri):
    if val > UCL or val < LCL:
      diem_mat_kiem_soat.append((idx, val))

  # Kịch bản 5: 2/3 điểm liên tục vượt ngoài ±2σ
  for i in range(len(diem_gia_tri) - 2):
    window = diem_gia_tri[i:i+3]
    idx_window = list(range(i, i+3))
    count_tren = sum([1 for v in window if v > mean + 2*sigma])
    count_duoi = sum([1 for v in window if v < mean - 2*sigma])

    if count_tren >= 2:
      for j, v in zip(idx_window, window):
        if v > mean + 2*sigma:
          diem_mat_kiem_soat.append((j, v))
      if count_tren >= 2:
        for j, v in zip(idx_window, window):
          diem_mat_kiem_soat.append((j, v))

    if count_duoi >= 2:
      for j, v in zip(idx_window, window):
        if v < mean - 2*sigma:
          diem_mat_kiem_soat.append((j, v))
      if count_duoi >= 2:
        for j, v in zip(idx_window, window):
          diem_mat_kiem_soat.append((j, v))

  # Loại bỏ trùng lặp điểm (nếu có nhiều window overlap)
  diem_mat_kiem_soat = list(set(diem_mat_kiem_soat))
  diem_mat_kiem_soat.sort(key=lambda x: x[0])

  return diem_mat_kiem_soat


# **VẼ BIỂU ĐỒ**

In [ ]:
def chart(chart_title, x_label, y_label, mean_label, data, diem_gia_tri, mean, UCL, LCL, label_diem_gia_tri, diem_mat_kiem_soat=None):
  plt.figure(figsize=(16,4))
  plt.plot(data.columns, diem_gia_tri, marker='o', linestyle='-', color='b', label=label_diem_gia_tri)
  plt.axhline(y=mean, color='g', linestyle='-')
  plt.axhline(y=UCL, color='r', linestyle='-')
  plt.axhline(y=LCL, color='r', linestyle='-')
  sigma = (UCL - mean) / 3
  for i in [1, 2]:
    plt.axhline(y=mean + i*sigma, color='gray', linestyle=':')
    plt.axhline(y=mean - i*sigma, color='gray', linestyle=':')

  # Đánh dấu điểm mất kiểm soát màu đỏ
  for idx, val in diem_mat_kiem_soat:
    plt.scatter(data.columns[idx], val, color='red', s=40, zorder=5, label="Điểm mất kiểm soát" if idx==diem_mat_kiem_soat[0][0] else "")
    plt.axvline(x=idx, color='gray', linestyle='--', alpha=0.5)

  xmax = len(diem_gia_tri) + 0.5
  plt.text(xmax, mean, mean_label, va='center', ha='left', color='g')
  plt.text(xmax, UCL, ' UCL', va='center', ha='left', color='r')
  plt.text(xmax, LCL, ' LCL', va='center', ha='left', color='r')
  plt.title(chart_title)
  plt.xlabel(x_label)
  plt.ylabel(y_label)
  plt.legend()
  plt.show()

# **CHỈ SỐ NĂNG LỰC** #

In [ ]:
def processCapability(data, subgroup_size=None, diem_mat_kiem_soat=None, diem_mat_kiem_soat_2=None):
  #USL = float(input("USL: "))
  #LSL = float(input("LSL: "))

  USL = 605
  LSL = 595

  # Loại bỏ điểm mất kiểm soá
  idx_set = set(idx for idx, _ in (diem_mat_kiem_soat or []))
  if diem_mat_kiem_soat_2:
    idx_set |= set(idx for idx, _ in diem_mat_kiem_soat_2)
  cols_to_exclude = [data.columns[i] for i in sorted(idx_set) if 0 <= i < data.shape[1]]
  if cols_to_exclude:
    data = data.drop(cols_to_exclude, axis=1, errors="ignore")
  print(data)

  # Tính trung bình tổng thể mu
  subgroup_mean = []
  subgroup_mean.append(data.mean())
  grand_mean = np.mean(subgroup_mean)

  # Tính độ lệch chuẩn của từng phân nhóm
  subgroup_std = data.std(ddof=1)
  Sbar = np.mean(subgroup_std)
  sigma_within = Sbar / constant.loc[subgroup_size, "c4"]

  # Tính độ lệch chuẩn tổng thể
  all_values = data.values.flatten()
  sigma_overall = all_values.std(ddof=1)

  # Tính Cp, Cpk
  Cp = (USL-LSL)/(6*sigma_within)
  CPU = (USL - grand_mean) / (3*sigma_within)
  CPL = (grand_mean - LSL) / (3*sigma_within)
  Cpk = min(CPU, CPL)
  print(f"Cp: {Cp}, Cpk: {Cpk}")

  # Tính Pp, Ppk
  Pp = (USL-LSL)/(6*sigma_overall)
  PPU = (USL - grand_mean) / (3*sigma_overall)
  PPL = (grand_mean - LSL) / (3*sigma_overall)
  Ppk = min(PPU, PPL)
  print(f"Pp: {Pp}, Ppk: {Ppk}")

  # Tính mức sigma
  #Z_LSL = (grand_mean-LSL)/sigma_within
  #Z_USL = (USL - grand_mean)/sigma_within

# **CÁC LOẠI BIỂU ĐỒ**

### Biểu đồ I-MR ###

In [ ]:
# Biểu đồ
def I_MR_chart():
  mean = data.mean(axis=1).iloc[0]
  #print(mean)
  d2 = constant.loc[2, "d2"]
  d3 = constant.loc[2, "d3"]

  # tính moving range
  s = data.iloc[0]
  mr = s.diff().abs().iloc[1:]
  mr = [0] + mr.tolist()
  mr_bar = np.mean(mr)
  #print(mr_bar)
  sigma = mr_bar/d2

  # Tính đường giới hạn
  UCL_I = mean + 3*sigma
  LCL_I = mean - 3*sigma
  UCL_MR = mr_bar + 3*d3*sigma
  LCL_MR = max(mr_bar - 3*d3*sigma,0)
  diem_mat_kiem_soat = []
  chart("Biểu đồ I", "Ngày", "Giá trị trung bình", r' $\bar{X}$', data, s, mean, UCL_I, LCL_I, "Điểm giá trị", diem_mat_kiem_soat=xu_ly_kich_ban_mat_kiem_soat(s, UCL_I, LCL_I, mean, sigma, diem_mat_kiem_soat))
  # Vẽ R chart
  diem_mat_kiem_soat_MR = []
  chart("Biểu đồ MR", "Ngày", "Khoảng biến thiên động", r' $\bar{MR}$', data, mr, mr_bar, UCL_MR, LCL_MR, "Điểm giá trị", diem_mat_kiem_soat=xu_ly_kich_ban_mat_kiem_soat(mr, UCL_MR, LCL_MR, mr_bar, sigma, diem_mat_kiem_soat_MR))



### Biểu đồ $\bar{X-}R$ ###

In [ ]:
def X_R_chart(subgroup_size):
  subgroup_mean = []
  subgroup_mean.append(data.mean())
  #print("Giá trị trung bình mẫu của mỗi phân nhóm\n")
  #print(subgroup_mean)
  #print("\nGiá trị trung bình tổng thể\n")
  grand_mean = np.mean(subgroup_mean)
  #print(grand_mean)
  ''''''
  subgroup_mean = np.array(subgroup_mean).flatten()
  R = []
  R.append(data.max()-data.min())
  #print(R)
  Rbar = np.mean(R)
  #print(round(Rbar,3))
  ''''''
  R = np.array(R).flatten()
  d2 = constant.loc[subgroup_size, "d2"]
  d3 = constant.loc[subgroup_size, "d3"]
  sigma = Rbar/d2
  #print(round(sigma,3))

  # Tính đường giới hạn
  UCL_X = grand_mean + 3*sigma/np.sqrt(len(data))
  LCL_X = grand_mean - 3*sigma/np.sqrt(len(data))
  UCL_R = Rbar + 3*d3*sigma
  LCL_R = Rbar - 3*d3*sigma

  # Vẽ Xbar chart
  diem_mat_kiem_soat = []
  chart("Biểu đồ X", "Ngày", "Giá trị trung bình", r' $\bar{X}$', data, subgroup_mean, grand_mean, UCL_X, LCL_X, "Điểm giá trị", diem_mat_kiem_soat=xu_ly_kich_ban_mat_kiem_soat(subgroup_mean, UCL_X, LCL_X, grand_mean, sigma, diem_mat_kiem_soat))
  # Vẽ R chart
  diem_mat_kiem_soat_R = []
  chart("Biểu đồ R", "Ngày", "Khoảng biến thiên", r' $\bar{R}$', data, R, Rbar, UCL_R, LCL_R, "Điểm giá trị", diem_mat_kiem_soat=xu_ly_kich_ban_mat_kiem_soat(R, UCL_R, LCL_R, Rbar, sigma, diem_mat_kiem_soat_R))

  #test_mat_kiem_soat = []
  #xu_ly_kich_ban_mat_kiem_soat(subgroup_mean, UCL_X, LCL_X, grand_mean, sigma, test_mat_kiem_soat)

  processCapability(data, subgroup_size, diem_mat_kiem_soat, diem_mat_kiem_soat_R)

### Biểu đồ $\bar{X-}S$ ###

In [ ]:
def X_S_chart(subgroup_size):
  subgroup_mean = []
  subgroup_mean.append(data.mean())
  #print("Giá trị trung bình mẫu của mỗi phân nhóm\n")
  #print(subgroup_mean)
  #print("\nGiá trị trung bình tổng thể\n")
  grand_mean = np.mean(subgroup_mean)
  #print(grand_mean)
  subgroup_mean = np.array(subgroup_mean).flatten()

  S = []
  S.append(data.std())
  print(S)
  Sbar = np.mean(S)
  #print(round(Rbar,3))
  ''''''
  S = np.array(S).flatten()
  c4 = constant.loc[subgroup_size, "c4"]
  c5 = constant.loc[subgroup_size, "c5"]
  sigma = Sbar/c4
  print(round(sigma,3))

  # Tính đường giới hạn
  UCL_X = grand_mean + 3*sigma/np.sqrt(len(data))
  LCL_X = grand_mean - 3*sigma/np.sqrt(len(data))
  UCL_S = Sbar + 3*c5*sigma
  LCL_S = Sbar - 3*c5*sigma

  # Vẽ Xbar chart
  diem_mat_kiem_soat = []
  chart("Biểu đồ X", "Ngày", "Giá trị trung bình", r' $\bar{X}$', data, subgroup_mean, grand_mean, UCL_X, LCL_X, "Điểm giá trị", diem_mat_kiem_soat=xu_ly_kich_ban_mat_kiem_soat(subgroup_mean, UCL_X, LCL_X, grand_mean, sigma, diem_mat_kiem_soat))
  # Vẽ R chart
  diem_mat_kiem_soat_S = []
  chart("Biểu đồ S", "Ngày", "Độ lệch chuẩn", r' $\bar{S}$', data, S, Sbar, UCL_S, LCL_S, "Điểm giá trị", diem_mat_kiem_soat=xu_ly_kich_ban_mat_kiem_soat(S, UCL_S, LCL_S, Sbar, sigma, diem_mat_kiem_soat_S))

  processCapability(data, subgroup_size, diem_mat_kiem_soat, diem_mat_kiem_soat_S)

### Biểu đồ P ###

In [ ]:
def P_chart(data):
  data["P"] = data["Rejects"] / data["Sampled"]
  P = data["P"].values
  P_mean = np.mean(P)

  sigma = np.sqrt(P_mean * (1 - P_mean) / data["Sampled"])
  UCL = P_mean + 3 * sigma
  LCL = np.maximum(P_mean - 3 * sigma, 0)

  x = np.arange(1, len(data) + 1)
  # Vẽ biểu đồ
  plt.figure(figsize=(16, 4))
  plt.plot(x, P, marker='o', linestyle='-', color='b', label="Tỷ lệ lỗi")
  plt.plot(x, UCL, linestyle='-', color='r', label="UCL")
  plt.plot(x, LCL, linestyle='-', color='r', label="LCL")
  plt.axhline(y=P_mean, color='g', linestyle='-', label=r'$\bar{P}$')

  # Nhãn cuối (dịch sang phải)
  plt.text(x[-1] + 0.2, P_mean, r' $\bar{P}$', va='center', ha='left', color='g')
  plt.text(x[-1] + 0.2, UCL.iloc[-1], ' UCL', va='center', ha='left', color='r')
  plt.text(x[-1] + 0.2, LCL.iloc[-1], ' LCL', va='center', ha='left', color='r')

  #plt.legend(loc="center left", bbox_to_anchor=(1, 0.5))
  plt.title("Biểu đồ P")
  plt.xlabel("Ngày")
  plt.ylabel("Tỷ lệ")
  plt.xticks(x)  # đảm bảo hiển thị đủ số nguyên
  #plt.tight_layout()
  plt.show()

### Biểu đồ U ###

In [ ]:
def U_chart(data):
  data["U"] = data["Defects"] / data["Sampled"]
  U = data["U"].values
  U_mean = np.mean(U)

  sigma = np.sqrt(U_mean / data["Sampled"])
  UCL = U_mean + 3 * sigma
  LCL = np.maximum(U_mean - 3 * sigma, 0)

  x = np.arange(1, len(data) + 1)
  # Vẽ biểu đồ
  plt.figure(figsize=(16, 4))
  plt.plot(x, U, marker='o', linestyle='-', color='b', label="Tỷ lệ lỗi")
  plt.plot(x, UCL, linestyle='-', color='r', label="UCL")
  plt.plot(x, LCL, linestyle='-', color='r', label="LCL")
  plt.axhline(y=U_mean, color='g', linestyle='-', label=r'$\bar{P}$')

  # Nhãn cuối (dịch sang phải)
  plt.text(x[-1] + 0.2, U_mean, r' $\bar{U}$', va='center', ha='left', color='g')
  plt.text(x[-1] + 0.2, UCL.iloc[-1], ' UCL', va='center', ha='left', color='r')
  plt.text(x[-1] + 0.2, LCL.iloc[-1], ' LCL', va='center', ha='left', color='r')

  #plt.legend(loc="center left", bbox_to_anchor=(1, 0.5))
  plt.title("Biểu đồ U")
  plt.xlabel("Ngày")
  plt.ylabel("Tỷ lệ")
  plt.xticks(x)  # đảm bảo hiển thị đủ số nguyên
  #plt.tight_layout()
  plt.show()

# **CHẠY CHƯƠNG TRÌNH**

In [ ]:
# Chọn kiểu dữ liệu
while True:
  print("1. Dữ liệu định lượng")
  print("2. Dữ liệu định tính")
  data_type = int(input("Kiểu dữ liệu: "))
  if(data_type == 1):
    print("\n0. Không \n1. Có")
    sub_group_choice = int(input("Thu thập theo phân nhóm? "))
    if(sub_group_choice==0):
      I_MR_chart()
      break
    elif(sub_group_choice==1):
      #print("\nKích thước phân nhóm?")
      subgroup_size = int(input("\nNhập kích thước phân nhóm: "))
      if(subgroup_size<=8 and subgroup_size>=2):
        X_R_chart(subgroup_size)
        break
      elif(subgroup_size>8):
        X_S_chart(subgroup_size)
        break
    break
  elif(data_type == 2):
    print("\n1. Số lượng khuyết tật\n2. Số lượng khuyết tật trên mỗi đơn vị")
    choice = int(input("Bạn đang đếm cái gì? "))
    if(choice == 1):
      P_chart(data)
      break
    elif(choice == 2):
      U_chart(data)
      break
  else:
    print("Kiểu dữ liệu không hợp lệ. \nNhập lại\n")